In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simpson

# Load both fighter and fight stat datasets
fighter_df = pd.read_csv("../Data Cleaning/cleaned data/cleaned_ufc_fighter_data.csv")
stat_df = pd.read_csv("../Data Cleaning/cleaned data/cleaned_ufc_fight_stat_data.csv")

# Merge on fighter_id
merged_df = pd.merge(stat_df, fighter_df, on="fighter_id")

# Filter for realistic UFC weights
merged_df = merged_df[(merged_df['fighter_weight_lbs'] >= 115) & (merged_df['fighter_weight_lbs'] <= 265)]

# Drop rows with missing values for takedown analysis
merged_df = merged_df.dropna(subset=['fighter_weight_lbs', 'takedown_succ', 'takedown_att'])
merged_df = merged_df[merged_df['takedown_att'] > 0]  # Remove cases with 0 attempts

# Compute takedown success rate
merged_df['takedown_success_rate'] = merged_df['takedown_succ'] / merged_df['takedown_att']
merged_df = merged_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['takedown_success_rate'])

# Bin weights
bins = np.arange(115, 270, 10)
merged_df['weight_bin'] = pd.cut(merged_df['fighter_weight_lbs'], bins)

# Group by weight bin
grouped = merged_df.groupby('weight_bin')['takedown_success_rate'].mean().reset_index()
grouped['weight_mid'] = grouped['weight_bin'].apply(lambda x: x.mid)

# Apply Simpson’s rule
x = grouped['weight_mid'].values
y = grouped['takedown_success_rate'].values
area = simpson(y, x)
print("Area under curve (Simpson’s Rule):", area)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(x, y, marker='o', linestyle='-', color='navy', label='Avg Takedown Success Rate')
plt.title("Average Takedown Success Rate by Fighter Weight")
plt.xlabel("Weight (lbs)")
plt.ylabel("Takedown Success Rate")
plt.grid(True)
plt.legend()
plt.savefig("figures/takedown_success_vs_weight.png", dpi=300)
plt.show()


/tmp/ipykernel_45486/114472553.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = merged_df.groupby('weight_bin')['takedown_success_rate'].mean().reset_index()


TypeError: simpson() takes 1 positional argument but 2 were given